In [30]:
import bs4 as BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
main_url = 'https://www.basketball-reference.com/players/'
#It was not seen prior that there are no players whos surnames start with x, hence an exception was thrown at letter x. 
#To save time, the available array was copied to another array and the remaining letters were processed 
#letters = 'abcdefghijklmnopqrstuvw'
letters = 'yz'
letters_list = list(letters)

In [15]:
""" htmldata = urllib.request.urlopen('https://www.basketball-reference.com/players/a/')
bsoup = BeautifulSoup.BeautifulSoup(htmldata,'html.parser')
complete_array = None
Data_list = None """
main_array = None
for letter in letters_list:
    print(letter)
    try:
        htmldata = urllib.request.urlopen('https://www.basketball-reference.com/players/' + letter +'/')
    except:
        continue
    bsoup = BeautifulSoup.BeautifulSoup(htmldata,'html.parser')
    complete_array = None
    Data_list = None
    for i in range(1,len(bsoup.find_all('tr'))):
        #we collect only players who played after 1980
        if int(bsoup.find_all('tr')[i].find('td',{'data-stat':'year_max'}).get_text())>1980:
            href = bsoup.find_all('tr')[i].find_all('a')[0].attrs['href']
            try:
                player_html = urllib.request.urlopen(main_url + letter + "/" + href.split('/')[-1])
            except:
                #due to errors in the html page, some href do not work hence we skip these
                continue
            player_bsoup = BeautifulSoup.BeautifulSoup(player_html,'html.parser')
            if Data_list == None:
                Data_list = [item.get_text() for item in player_bsoup.find_all('th') if 'aria-label' in item.attrs.keys()]
                Data_list.insert(0,'Name')
            player_dataarray = np.empty(shape = (len(player_bsoup.find_all('tbody')[0].find_all('tr')),len(Data_list)), dtype = np.object)
            for j in range(0,len(player_bsoup.find_all('tbody')[0].find_all('tr'))):
                data_row = [player_bsoup.find_all('h1',{'itemprop':'name'})[0].get_text().strip()]
                #getting season
                try:
                    #some players wont play some years, at those instances, this code will throw exception
                    #hence we continue to the next year
                    data_row.extend([player_bsoup.find_all('tbody')[0].find_all('tr')[j].find_all('th')[0].get_text()])
                    #adding rest of data    
                    [data_row.append(data) for data in [td.get_text() if td.get_text() != "" else np.nan for td in (player_bsoup.find_all                     ('tbody')[0].find_all('tr')[j].find_all('td'))]]    
                except:
                    continue
                try:     
                    player_dataarray[j,:]  = np.array(data_row)
                except:
                    #any player who doesnt have ['Name','Season','Age','Tm','Lg','Pos','G','GS','MP','FG','FGA','FG%','3P','3PA','3P%', '2P''2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS'] as there stats are not considered. Majority of players after 1980 will have these stats as these were collected and recorded in this era of basketball
                    break
        if i == 1:
            complete_array = player_dataarray
        else:
            complete_array = np.vstack((complete_array,player_dataarray))
    if letter == letters_list[0]:
        main_array = complete_array
        print(main_array.shape)
    else:
        main_array = np.vstack((main_array,complete_array))
        print(main_array.shape)
        
    


y
(87, 31)
z
(184, 31)


In [16]:
y_to_z_dataarray = main_array.copy()

In [13]:
a_to_w_dataarray = main_array.copy()

In [14]:
a_to_w_dataarray

array([['Alaa Abdelnaby', '1990-91', '22', ..., '0.5', '0.9', '3.1'],
       ['Alaa Abdelnaby', '1991-92', '23', ..., '0.9', '1.9', '6.1'],
       ['Alaa Abdelnaby', '1992-93', '24', ..., '1.3', '2.5', '7.7'],
       ...,
       ['Tony Wroten', '2014-15', '21', ..., '3.8', '2.4', '16.9'],
       ['Tony Wroten', '2015-16', '22', ..., '3.6', '1.4', '8.4'],
       ['A.J. Wynder', '1990-91', '26', ..., '0.7', '0.2', '2.0']],
      dtype=object)

In [17]:
final_array = np.vstack((a_to_w_dataarray, y_to_z_dataarray))
final_array

array([['Alaa Abdelnaby', '1990-91', '22', ..., '0.5', '0.9', '3.1'],
       ['Alaa Abdelnaby', '1991-92', '23', ..., '0.9', '1.9', '6.1'],
       ['Alaa Abdelnaby', '1992-93', '24', ..., '1.3', '2.5', '7.7'],
       ...,
       ['Ivica Zubac', '2018-19', '21', ..., '1.4', '2.5', '9.4'],
       ['Ivica Zubac', '2019-20', '22', ..., '0.8', '2.3', '8.3'],
       ['Ivica Zubac', '2020-21', '23', ..., '0.9', '2.4', '7.6']],
      dtype=object)

In [26]:
final_df = pd.DataFrame(data = final_array, columns = Data_list)
final_df

,Name,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Alaa Abdelnaby,1990-91,22,POR,NBA,PF,43,0,6.7,1.3,...,.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1
1,Alaa Abdelnaby,1991-92,23,POR,NBA,PF,71,1,13.2,2.5,...,.752,1.1,2.5,3.7,0.4,0.4,0.2,0.9,1.9,6.1
2,Alaa Abdelnaby,1992-93,24,TOT,NBA,PF,75,52,17.5,3.3,...,.759,1.7,2.8,4.5,0.4,0.3,0.3,1.3,2.5,7.7
3,Alaa Abdelnaby,1992-93,24,MIL,NBA,PF,12,0,13.3,2.2,...,.750,1.0,2.1,3.1,0.8,0.5,0.3,1.1,2.0,5.3
4,Alaa Abdelnaby,1992-93,24,BOS,NBA,PF,63,52,18.3,3.5,...,.760,1.8,3.0,4.8,0.3,0.3,0.3,1.3,2.6,8.2
5,Alaa Abdelnaby,1993-94,25,BOS,NBA,PF,13,0,12.2,1.8,...,.640,0.9,2.6,3.5,0.2,0.2,0.2,1.3,1.5,4.9
6,Alaa Abdelnaby,1994-95,26,TOT,NBA,PF,54,0,9.4,2.2,...,.571,0.7,1.4,2.1,0.2,0.3,0.2,0.8,1.9,4.7
7,Alaa Abdelnaby,1994-95,26,SAC,NBA,PF,51,0,9.3,2.3,...,.571,0.7,1.4,2.1,0.3,0.3,0.2,0.8,2.0,5.0
8,Alaa Abdelnaby,1994-95,26,PHI,NBA,PF,3,0,10.0,0.3,...,nan,1.0,1.7,2.7,0.0,0.0,0.0,1.7,0.7,0.7
9,Alaa Abdelnaby,1990-91,22,POR,NBA,PF,43,0,6.7,1.3,...,.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1


In [29]:
final_df.to_csv('NBAData.csv',index = False)